In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler

In [38]:
train=pd.read_csv('./Dataset/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('./Dataset/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [39]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
'자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [40]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [41]:
X_train_sc = X_train
X_test_sc = X_test

In [42]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [43]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def perform_rf_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # RandomForest hyperparameter candidate list setup
    param_grid = {
        'n_estimators': [100, 200, 300,500],
        'max_depth': [5, 10,15],
        'random_state': [0]
    }

    # Initialize a RandomForest classifier
    model = RandomForestClassifier()

    # Grid search setup
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # Fit the model and tune
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters
    print("Best Hyperparameters:", grid_search.best_params_)

    # Calculate the average evaluation metric
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [44]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def evaluate_rf_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # Initialize a RandomForest model
    model = RandomForestClassifier(**best_params)

    # Lists to save the evaluation metrics for each fold
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # Fit the model
        model.fit(X_train_fold, y_train_fold)

        # Get the predicted probabilities on the test data
        probabilities = model.predict_proba(X_test_fold)

        # Adjust the predicted classes with a threshold of 0.5
        threshold = 0.46
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # Append the evaluation metrics for each fold to the lists
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # Save the model with the best F1 score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # Perform the final prediction with the model with the best F1 score
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # Calculate the evaluation metrics
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [45]:
best_params, mean_f1_score = perform_rf_grid_search(X_train, y_train, k_fold=5)
evaluate_rf_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


c:\Users\dgh06\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Best Hyperparameters: {'max_depth': 15, 'n_estimators': 200, 'random_state': 0}
Mean F1 Score: 0.596467422858988


C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_fold, y_train_fold)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:28: DataConversionWarning: A column-vector y 

Fold 1
Accuracy: 0.821917808219178
Precision: 0.6666666666666666
Recall: 0.5555555555555556
F1 score: 0.606060606060606
Confusion Matrix:
[[50  5]
 [ 8 10]]
------------------------------


...

Fold 3
Accuracy: 0.8194444444444444
Precision: 0.7272727272727273
Recall: 0.4444444444444444
F1 score: 0.5517241379310345
Confusion Matrix:
[[51  3]
 [10  8]]
------------------------------


C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_fold, y_train_fold)


Fold 4
Accuracy: 0.875
Precision: 0.7368421052631579
Recall: 0.7777777777777778
F1 score: 0.7567567567567567
Confusion Matrix:
[[49  5]
 [ 4 14]]
------------------------------
Fold 5
Accuracy: 0.8472222222222222
Precision: 0.7058823529411765
Recall: 0.6666666666666666
F1 score: 0.6857142857142857
Confusion Matrix:
[[49  5]
 [ 6 12]]
------------------------------
Final Test Results
Accuracy: 0.8611111111111112
Precision: 0.75
Recall: 0.6666666666666666
F1 score: 0.7058823529411765
Confusion Matrix:
[[75  6]
 [ 9 18]]


C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:35: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)
C:\Users\dgh06\AppData\Local\Temp\ipykernel_7664\2993980058.py:67: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

([0.821917808219178,
  0.7534246575342466,
  0.8194444444444444,
  0.875,
  0.8472222222222222],
 [0.6666666666666666,
  0.5,
  0.7272727272727273,
  0.7368421052631579,
  0.7058823529411765],
 [0.5555555555555556,
  0.4444444444444444,
  0.4444444444444444,
  0.7777777777777778,
  0.6666666666666666],
 [0.606060606060606,
  0.47058823529411764,
  0.5517241379310345,
  0.7567567567567567,
  0.6857142857142857])

param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 5, 10],
        'random_state': [0]}
- 0.5909